In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import plotly.express as px
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import yfinance as yf

In [2]:
# Get data from yahoo finance
df = yf.download('GLD', start='2023-01-01')
df.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-08-17,176.529999,176.559998,174.929993,175.300003,175.300003,7937700
2023-08-18,175.830002,176.059998,175.100006,175.330002,175.330002,6251000
2023-08-21,175.899994,175.940002,174.960007,175.809998,175.809998,4206400
2023-08-22,175.830002,176.250000,175.309998,176.100006,176.100006,5892200
2023-08-23,177.320007,178.070007,177.074997,177.824997,177.824997,3729343


In [3]:
# plot
px.line(df, x=df.index, y='Close')

In [4]:
# Log Transform
df['Log-Close'] = np.log(df['Close'])

In [5]:
# plot
px.line(df, x=df.index, y='Log-Close')

In [6]:
# Split data into train and test
n_test = 30
train = df.iloc[: -n_test]
test = df.iloc[-n_test:]

In [7]:
# Set frequency to 'business day' not working
# df.index.freq = 'B'

In [8]:
# Instantiation
model = ExponentialSmoothing(train['Log-Close'], trend='add', seasonal=None, initialization_method='legacy-heuristic')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [9]:
# Fit the model
res = model.fit()

In [10]:
# Set index for 'Holt-Winters' Column
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

In [11]:
# Calculate Train value and Forecast value for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters-Train'] = res.fittedvalues
df.loc[test_idx, 'Holt-Winters-Test'] = res.forecast(n_test).to_numpy()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [12]:
# Plot
px.line(df, x=df.index, y=['Log-Close', 'Holt-Winters-Train', 'Holt-Winters-Test'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 161 entries, 2023-01-03 to 2023-08-23
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                161 non-null    float64
 1   High                161 non-null    float64
 2   Low                 161 non-null    float64
 3   Close               161 non-null    float64
 4   Adj Close           161 non-null    float64
 5   Volume              161 non-null    int64  
 6   Log-Close           161 non-null    float64
 7   Holt-Winters-Train  131 non-null    float64
 8   Holt-Winters-Test   30 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 12.6 KB
